<a href="https://colab.research.google.com/github/Chamara-g/pre-datastorm-v1-0/blob/master/pre_data_storm_NN_design.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import numpy as np
import pandas as pd

In [3]:
#read train data set
df = pd.read_csv("drive/My Drive/FYP/dataset/credit_card_data_pre.csv", index_col=0)
df.head()

,Balance_Limit_V1,Gender,EDUCATION_STATUS,MARITAL_STATUS,AGE,PAY_JULY,PAY_AUG,PAY_SEP,PAY_OCT,PAY_NOV,PAY_DEC,DUE_AMT_JULY,DUE_AMT_AUG,DUE_AMT_SEP,DUE_AMT_OCT,DUE_AMT_NOV,DUE_AMT_DEC,PAID_AMT_JULY,PAID_AMT_AUG,PAID_AMT_SEP,PAID_AMT_OCT,PAID_AMT_NOV,PAID_AMT_DEC,DUE_AMT_GRD,PAID_AMT_GRD,PAY_GRD,NEXT_MONTH_DEFAULT
Client_ID,,,,,,,,,,,,,,,,,,,,,,,,,,,
A2,1000000.0,1,2,1,2,-1,-1,-1,-1,-1,-1,3248,3389,6004,39418,162772,-13982,3437,6004,39418,162772,0,538165,12154.657143,79399.485714,-6,0
A3,1000000.0,0,1,1,3,0,-1,-1,-1,-1,0,353351,151818,26948,43530,80811,124590,151818,46200,43530,80811,942,33666,-38292.685714,-19692.942857,-4,0
A4,100000.0,0,1,0,2,4,3,2,2,-2,-2,16681,16082,15477,0,0,0,0,0,0,0,0,0,-4203.657143,-0.000000,7,1
A5,200000.0,0,2,0,2,2,0,0,0,0,0,90457,92848,95193,97309,100353,102740,3855,3890,3696,4620,4049,3918,2458.457143,49.028571,2,1
A6,1000000.0,0,2,1,2,2,2,0,0,0,0,429556,419466,429785,435354,445271,453899,0,20790,16170,17325,16401,17325,5848.542857,2131.800000,4,0


In [7]:
#read test data set
df_test = pd.read_csv("drive/My Drive/FYP/dataset/credit_card_data_test_pre.csv", index_col=0)
df_test.head()

,Balance_Limit_V1,Gender,EDUCATION_STATUS,MARITAL_STATUS,AGE,PAY_JULY,PAY_AUG,PAY_SEP,PAY_OCT,PAY_NOV,PAY_DEC,DUE_AMT_JULY,DUE_AMT_AUG,DUE_AMT_SEP,DUE_AMT_OCT,DUE_AMT_NOV,DUE_AMT_DEC,PAID_AMT_JULY,PAID_AMT_AUG,PAID_AMT_SEP,PAID_AMT_OCT,PAID_AMT_NOV,PAID_AMT_DEC,DUE_AMT_GRD,PAID_AMT_GRD,PAY_GRD,PAY_DUE_GRD
Client_ID,,,,,,,,,,,,,,,,,,,,,,,,,,,
A20170,500000.0,1,0,0,2,0,0,0,0,0,0,194188,198510,128316,311201,351790,352206,13710,12816,195701,53246,17256,12991,40937.571429,-3792.285714,0,18572.642857
A16887,200000.0,0,0,0,2,0,0,0,0,0,0,87115,89427,91823,94181,97221,100275,3756,3842,3890,4620,4620,4620,2615.428571,210.971429,0,1413.200000
A3430,400000.0,1,0,1,3,-1,-1,2,0,0,-1,2693,9573,8984,11294,9561,993,6930,0,2310,0,2310,40060,-177.885714,4864.857143,-1,2343.485714
A3696,1500000.0,1,2,0,2,1,-1,-1,-2,-2,-2,0,5775,5775,0,0,0,5775,5775,0,0,0,0,-660.000000,-1320.000000,-7,-990.000000
A20474,100000.0,0,1,1,3,0,0,-2,-2,-2,-2,85257,0,0,0,0,0,0,0,0,0,0,0,-12179.571429,-0.000000,-8,-6089.785714


In [0]:
#test data set pre processing

#convet to string balance limit to float
df_test.Balance_Limit_V1 = (df_test.Balance_Limit_V1.replace(r'[KMB]+$', '', regex=True).astype(float) * df_test.Balance_Limit_V1.str.extract(r'[\d\.]+([KMB]+)', expand=False).fillna(1).replace(['K','M', 'B'], [10**3, 10**6, 10**9]).astype(int))
df_test.head()

#gender education marital age stringto int convert
df_test.Gender.replace(['M', 'F'], [1, 0], inplace=True)
df_test.EDUCATION_STATUS.replace(['Graduate', 'High School', 'Other'], [2, 1, 0], inplace=True)
df_test.MARITAL_STATUS.replace(['Other', 'Single'], [1, 0], inplace=True)
df_test.AGE.replace(['Less than 30', '31-45', '46-65','More than 65'], [3, 2, 1, 0], inplace=True)

df_test.head()

,Balance_Limit_V1,Gender,EDUCATION_STATUS,MARITAL_STATUS,AGE,PAY_JULY,PAY_AUG,PAY_SEP,PAY_OCT,PAY_NOV,PAY_DEC,DUE_AMT_JULY,DUE_AMT_AUG,DUE_AMT_SEP,DUE_AMT_OCT,DUE_AMT_NOV,DUE_AMT_DEC,PAID_AMT_JULY,PAID_AMT_AUG,PAID_AMT_SEP,PAID_AMT_OCT,PAID_AMT_NOV,PAID_AMT_DEC
Client_ID,,,,,,,,,,,,,,,,,,,,,,,
A20170,500000.0,1,0,0,2,0,0,0,0,0,0,194188,198510,128316,311201,351790,352206,13710,12816,195701,53246,17256,12991
A16887,200000.0,0,0,0,2,0,0,0,0,0,0,87115,89427,91823,94181,97221,100275,3756,3842,3890,4620,4620,4620
A3430,400000.0,1,0,1,3,-1,-1,2,0,0,-1,2693,9573,8984,11294,9561,993,6930,0,2310,0,2310,40060
A3696,1500000.0,1,2,0,2,1,-1,-1,-2,-2,-2,0,5775,5775,0,0,0,5775,5775,0,0,0,0
A20474,100000.0,0,1,1,3,0,0,-2,-2,-2,-2,85257,0,0,0,0,0,0,0,0,0,0,0


In [0]:
#train data set pre processing

#convet to string balance limit to float
df.Balance_Limit_V1 = (df.Balance_Limit_V1.replace(r'[KMB]+$', '', regex=True).astype(float) * df.Balance_Limit_V1.str.extract(r'[\d\.]+([KMB]+)', expand=False).fillna(1).replace(['K','M', 'B'], [10**3, 10**6, 10**9]).astype(int))
df.head()

#gender education marital age stringto int convert
df.Gender.replace(['M', 'F'], [1, 0], inplace=True)
df.EDUCATION_STATUS.replace(['Graduate', 'High School', 'Other'], [2, 1, 0], inplace=True)
df.MARITAL_STATUS.replace(['Other', 'Single'], [1, 0], inplace=True)
df.AGE.replace(['Less than 30', '31-45', '46-65','More than 65'], [3, 2, 1, 0], inplace=True)

df.head()

,Balance_Limit_V1,Gender,EDUCATION_STATUS,MARITAL_STATUS,AGE,PAY_JULY,PAY_AUG,PAY_SEP,PAY_OCT,PAY_NOV,PAY_DEC,DUE_AMT_JULY,DUE_AMT_AUG,DUE_AMT_SEP,DUE_AMT_OCT,DUE_AMT_NOV,DUE_AMT_DEC,PAID_AMT_JULY,PAID_AMT_AUG,PAID_AMT_SEP,PAID_AMT_OCT,PAID_AMT_NOV,PAID_AMT_DEC,NEXT_MONTH_DEFAULT
Client_ID,,,,,,,,,,,,,,,,,,,,,,,,
A2,1000000.0,1,2,1,2,-1,-1,-1,-1,-1,-1,3248,3389,6004,39418,162772,-13982,3437,6004,39418,162772,0,538165,0
A3,1000000.0,0,1,1,3,0,-1,-1,-1,-1,0,353351,151818,26948,43530,80811,124590,151818,46200,43530,80811,942,33666,0
A4,100000.0,0,1,0,2,4,3,2,2,-2,-2,16681,16082,15477,0,0,0,0,0,0,0,0,0,1
A5,200000.0,0,2,0,2,2,0,0,0,0,0,90457,92848,95193,97309,100353,102740,3855,3890,3696,4620,4049,3918,1
A6,1000000.0,0,2,1,2,2,2,0,0,0,0,429556,419466,429785,435354,445271,453899,0,20790,16170,17325,16401,17325,0


**NN model Train**

In [6]:
# first neural network with keras tutorial
from numpy import loadtxt
from keras.models import Sequential
from keras.layers import Dense
import tensorflow.compat.v1 as tf

from tensorflow import keras
import tensorflow.compat.v1 as tf

# split into input (X) and output (y) variables
X = df.drop("NEXT_MONTH_DEFAULT",1)   #Feature Matrix
y = df["NEXT_MONTH_DEFAULT"]          #Target Variable

selected_col_names = ["PAY_JULY",	"PAY_AUG",	"PAY_SEP",	"PAY_GRD"]
#selected_col_names = ["PAY_JULY" ,"PAID_AMT_JULY" ,"PAY_AUG" ,"PAID_AMT_AUG" ,"PAY_SEP" ,"PAID_AMT_SEP" ,"PAY_OCT" ,"PAID_AMT_OCT" ,"PAY_NOV" ,"PAID_AMT_NOV" ,"PAY_DEC" ,"PAID_AMT_DEC"]
X_Select = X[selected_col_names]

# define the keras model
#model = Sequential()
#model.add(Dense(12, input_dim=4, activation='relu'))
#model.add(Dense(4, activation='relu'))
#model.add(Dense(1, activation='sigmoid'))

model = keras.Sequential([
    keras.layers.Flatten(input_shape=(4,)),
    keras.layers.Dense(10, activation=tf.nn.relu),
	  keras.layers.Dense(10, activation=tf.nn.sigmoid),
	  keras.layers.Dense(10, activation=tf.nn.relu),
    keras.layers.Dense(1, activation=tf.nn.sigmoid),
])

# compile the keras model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# fit the keras model on the dataset
model.fit(X_Select, y, epochs=150, batch_size=10)

# evaluate the keras model
_, accuracy = model.evaluate(X_Select, y)
print('Accuracy: %.2f' % (accuracy*100))

Train on 24000 samples
Epoch 1/150
24000/24000 [==============================] - 7s 290us/sample - loss: 0.4605 - acc: 0.8055
Epoch 2/150
24000/24000 [==============================] - 7s 285us/sample - loss: 0.4455 - acc: 0.8188
Epoch 3/150
24000/24000 [==============================] - 7s 294us/sample - loss: 0.4442 - acc: 0.8190
Epoch 4/150
24000/24000 [==============================] - 7s 285us/sample - loss: 0.4439 - acc: 0.8198
Epoch 5/150
24000/24000 [==============================] - 7s 286us/sample - loss: 0.4435 - acc: 0.8195
Epoch 6/150
24000/24000 [==============================] - 7s 272us/sample - loss: 0.4430 - acc: 0.8195
Epoch 7/150
24000/24000 [==============================] - 7s 281us/sample - loss: 0.4429 - acc: 0.8203
Epoch 8/150
24000/24000 [==============================] - 7s 275us/sample - loss: 0.4428 - acc: 0.8198
Epoch 9/150
24000/24000 [==============================] - 6s 263us/sample - loss: 0.4422 - acc: 0.8209
Epoch 10/150
24000/24000 [===============

In [8]:
# make class predictions with the model

X_test = df_test[selected_col_names]
y_pred_NN = model.predict_classes(X_test)
df_test["NEXT_MONTH_DEFAULT"] = y_pred_NN
df_test.head()

,Balance_Limit_V1,Gender,EDUCATION_STATUS,MARITAL_STATUS,AGE,PAY_JULY,PAY_AUG,PAY_SEP,PAY_OCT,PAY_NOV,PAY_DEC,DUE_AMT_JULY,DUE_AMT_AUG,DUE_AMT_SEP,DUE_AMT_OCT,DUE_AMT_NOV,DUE_AMT_DEC,PAID_AMT_JULY,PAID_AMT_AUG,PAID_AMT_SEP,PAID_AMT_OCT,PAID_AMT_NOV,PAID_AMT_DEC,DUE_AMT_GRD,PAID_AMT_GRD,PAY_GRD,PAY_DUE_GRD,NEXT_MONTH_DEFAULT
Client_ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,
A20170,500000.0,1,0,0,2,0,0,0,0,0,0,194188,198510,128316,311201,351790,352206,13710,12816,195701,53246,17256,12991,40937.571429,-3792.285714,0,18572.642857,0
A16887,200000.0,0,0,0,2,0,0,0,0,0,0,87115,89427,91823,94181,97221,100275,3756,3842,3890,4620,4620,4620,2615.428571,210.971429,0,1413.200000,0
A3430,400000.0,1,0,1,3,-1,-1,2,0,0,-1,2693,9573,8984,11294,9561,993,6930,0,2310,0,2310,40060,-177.885714,4864.857143,-1,2343.485714,0
A3696,1500000.0,1,2,0,2,1,-1,-1,-2,-2,-2,0,5775,5775,0,0,0,5775,5775,0,0,0,0,-660.000000,-1320.000000,-7,-990.000000,0
A20474,100000.0,0,1,1,3,0,0,-2,-2,-2,-2,85257,0,0,0,0,0,0,0,0,0,0,0,-12179.571429,-0.000000,-8,-6089.785714,0


In [0]:
df_out = df_test['NEXT_MONTH_DEFAULT']
df_out.to_csv('drive/My Drive/FYP/dataset/credit_card_output_NN_NEW_F4_150.csv', encoding='utf-8', mode='w', header=["NEXT_MONTH_DEFAULT"], index=True)